## IDEA <br>

- Use OMDB API: http://www.omdbapi.com/#usage
- Do the API request with the search, movies with word 'love', extract info from x pages
- Like that get the list of titles of the movies
- Do another API request title by title to get more info about the movies
- Put info from both requests to one df
- Scrap IMDB or Wikipedia based on title or IMDB ID to get info from there
- Check if some data matching: Release year, Genre, Director, IMDB Rating
- Could also get more data about the director from Wiki or IMDB

## API <br>

- For the API portion of the project will need to make calls to your chosen API, successfully obtain a response, request data, convert it into a Pandas data frame, and export it as a CSV file.

In [162]:
from pandas.io.json import json_normalize
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re


#check if using this library
import time

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#the function to flatten_json several times
#for now haven't used them

def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

- Doing the search of the titles that include word 'love' in x pages

In [3]:
#generating several jsons from different pages
#in the range indicate how many pages want to check (API in general gives MAX movies per search)
#and then adding all the movie names to one list
#per each loop generating json for one page and extracting titles

url_pat= 'http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&s=love&type=movie&page='
final_list = []
for page in range(1,5):
    page = str(page)
    url = url_pat + page
    response = requests.get(url)
    results = response.json()
    title_box = results.get('Search')
    title_list = [title.get('Title') for title in title_box]
    final_list += title_list
final_list

['Crazy, Stupid, Love.',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb',
 'Love, Actually',
 'Shakespeare in Love',
 'P.S. I Love You',
 'I Love You, Man',
 'Love & Other Drugs',
 'Punch-Drunk Love',
 'Love, Rosie',
 'In the Mood for Love',
 'From Russia with Love',
 'From Paris with Love',
 'I Love You Phillip Morris',
 'Love, Simon',
 'Eat Pray Love',
 'To Rome with Love',
 'Stuck in Love.',
 'A Lot Like Love',
 'Love Me If You Dare',
 'Wristcutters: A Love Story',
 'The Love Guru',
 'Show Me Love',
 'New York, I Love You',
 'American Pie Presents: The Book of Love',
 'Love',
 'Capitalism: A Love Story',
 'Endless Love',
 'Down with Love',
 'Sea of Love',
 'Everyone Says I Love You',
 'Love and Death',
 'Love & Mercy',
 'I Love You, Beth Cooper',
 'The One I Love',
 "Can't Buy Me Love",
 'For Love of the Game',
 'All the Boys Love Mandy Lane',
 'Love Happens',
 'Must Love Dogs',
 'Love Story']

- converting names into the correct format to later pass them to the URL

In [5]:
titles_formatted = [string.replace(" ", "+") for string in final_list]
titles_formatted

['Crazy,+Stupid,+Love.',
 'Dr.+Strangelove+or:+How+I+Learned+to+Stop+Worrying+and+Love+the+Bomb',
 'Love,+Actually',
 'Shakespeare+in+Love',
 'P.S.+I+Love+You',
 'I+Love+You,+Man',
 'Love+&+Other+Drugs',
 'Punch-Drunk+Love',
 'Love,+Rosie',
 'In+the+Mood+for+Love',
 'From+Russia+with+Love',
 'From+Paris+with+Love',
 'I+Love+You+Phillip+Morris',
 'Love,+Simon',
 'Eat+Pray+Love',
 'To+Rome+with+Love',
 'Stuck+in+Love.',
 'A+Lot+Like+Love',
 'Love+Me+If+You+Dare',
 'Wristcutters:+A+Love+Story',
 'The+Love+Guru',
 'Show+Me+Love',
 'New+York,+I+Love+You',
 'American+Pie+Presents:+The+Book+of+Love',
 'Love',
 'Capitalism:+A+Love+Story',
 'Endless+Love',
 'Down+with+Love',
 'Sea+of+Love',
 'Everyone+Says+I+Love+You',
 'Love+and+Death',
 'Love+&+Mercy',
 'I+Love+You,+Beth+Cooper',
 'The+One+I+Love',
 "Can't+Buy+Me+Love",
 'For+Love+of+the+Game',
 'All+the+Boys+Love+Mandy+Lane',
 'Love+Happens',
 'Must+Love+Dogs',
 'Love+Story']

- Getting the info about each movie from the list

In [6]:
url_pattern = 'http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&t='
appended_dfs=[]

for title in titles_formatted:
    url = url_pattern + title
    response = requests.get(url)
    results = response.json()
    flattened_data = json_normalize(results)
    #appending dataframes to the list to later concatenate them to one dataframe
    appended_dfs.append(flattened_data)
    love_movies = pd.concat(appended_dfs, ignore_index = True) #ignoring index, to have 0,1,2...n, if not, get 0,0,0,...

love_movies

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,"Crazy, Stupid, Love.",2011,PG-13,29 Jul 2011,118 min,"Comedy, Drama, Romance","Glenn Ficarra, John Requa",Dan Fogelman,"Steve Carell, Ryan Gosling, Julianne Moore, Em...",A middle-aged husband's life changes dramatica...,...,68,7.4,"447,664",tt1570728,movie,01 Nov 2011,"$83,300,000",Warner Bros. Pictures,N/A,True
1,Dr. Strangelove or: How I Learned to Stop Worr...,1964,PG,29 Jan 1964,95 min,Comedy,Stanley Kubrick,"Stanley Kubrick (screenplay), Terry Southern (...","Peter Sellers, George C. Scott, Sterling Hayde...",An insane general triggers a path to nuclear h...,...,97,8.4,"427,212",tt0057012,movie,02 Nov 2004,N/A,Sony Pictures,N/A,True
2,"Love, Actually",2003,R,14 Nov 2003,135 min,"Comedy, Drama, Romance",Richard Curtis,Richard Curtis,"Bill Nighy, Gregor Fisher, Rory MacGregor, Col...",Follows the lives of eight very different coup...,...,55,7.6,"405,898",tt0314331,movie,27 Apr 2004,"$59,365,105",Universal Pictures,N/A,True
3,Shakespeare in Love,1998,R,08 Jan 1999,123 min,"Comedy, Drama, History, Romance",John Madden,"Marc Norman, Tom Stoppard","Geoffrey Rush, Tom Wilkinson, Steven O'Donnell...","The world's greatest ever playwright, William ...",...,87,7.1,"205,961",tt0138097,movie,10 Aug 1999,N/A,Miramax,N/A,True
4,P.S. I Love You,2007,PG-13,21 Dec 2007,126 min,"Comedy, Drama, Romance",Richard LaGravenese,"Richard LaGravenese (screenplay), Steven Roger...","Hilary Swank, Gerard Butler, Lisa Kudrow, Gina...",A young widow discovers that her late husband ...,...,39,7.0,"194,541",tt0431308,movie,05 Feb 2008,N/A,N/A,N/A,True
5,"I Love You, Man",2009,R,20 Mar 2009,105 min,"Comedy, Romance",John Hamburg,"John Hamburg (screenplay), Larry Levin (screen...","Paul Rudd, Rashida Jones, Sarah Burns, Greg Le...",Friendless Peter Klaven goes on a series of ma...,...,70,7.0,"188,613",tt1155056,movie,11 Aug 2009,"$71,300,000",Dreamworks/Paramount,N/A,True
6,Love,2015,Not Rated,30 Oct 2015,135 min,"Drama, Romance",Gaspar Noé,Gaspar Noé,"Aomi Muyock, Karl Glusman, Klara Kristin, Ugo Fox",Murphy is an American living in Paris who ente...,...,51,6.1,"40,472",tt3774694,movie,15 Mar 2016,"$176,061",Alchemy,N/A,True
7,Punch-Drunk Love,2002,R,01 Nov 2002,95 min,"Comedy, Drama, Romance, Thriller",Paul Thomas Anderson,Paul Thomas Anderson,"Adam Sandler, Jason Andrews, Don McManus, Emil...",A psychologically troubled novelty supplier is...,...,78,7.3,"137,629",tt0272338,movie,24 Jun 2003,"$17,800,000",Columbia Pictures,N/A,True
8,"Love, Rosie",2014,R,22 Oct 2014,102 min,"Comedy, Romance",Christian Ditter,"Juliette Towhidi (screenplay), Cecelia Ahern (...","Lily Collins, Sam Claflin, Christian Cooke, Ja...",Rosie and Alex have been best friends since th...,...,44,7.2,"111,376",tt1638002,movie,05 May 2015,N/A,Film Arcade,N/A,True
9,In the Mood for Love,2000,PG,09 Mar 2001,98 min,"Drama, Romance",Kar-Wai Wong,Kar-Wai Wong,"Maggie Cheung, Tony Chiu-Wai Leung, Ping Lam S...","Two neighbors, a woman and a man, form a stron...",...,85,8.1,"111,148",tt0118694,movie,05 Mar 2002,N/A,USA Films,N/A,True


In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
#??????? maybe need to flatten again to get the info from the ratings column

#check how influences the release date being closer to valentines day (check in number of days..) to the rating, 
#if there is correlation

#could create another df with title and ratings from different sources, choose which source to scrap

#BUT

#have IMDB rating already in another column extracted

In [8]:
love_movies.head()

#from wiki table: 
#title, released,runtime, director, writer, actors, language, country, box office (see that not matching) 
#but it's matchin to gross worldwide from IMDB, production

#will need to delet last two columns: website and Response
#from wiki can add production company, because in API under production have distribution company AND budget
#can update some NAs from wiki, like box office

#url to use: https://en.wikipedia.org/wiki/Crazy,_Stupid,_Love
#will need to format again the names





#from IMDB
#search by this url: https://www.imdb.com/title/tt1570728/
#could check the main info, need to select which info going to check in wiki, api and imdb
#could extract quates from imdb and add them to the df, let's say we will need to use them for the MKT materials


,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,"Crazy, Stupid, Love.",2011,PG-13,29 Jul 2011,118 min,"Comedy, Drama, Romance","Glenn Ficarra, John Requa",Dan Fogelman,"Steve Carell, Ryan Gosling, Julianne Moore, Emma Stone","A middle-aged husband's life changes dramatically when his wife asks him for a divorce. He seeks to rediscover his manhood with the help of a newfound friend, Jacob, learning to pick up girls at bars.",English,USA,Nominated for 1 Golden Globe. Another 3 wins & 22 nominations.,https://m.media-amazon.com/images/M/MV5BMTg2MjkwMTM0NF5BMl5BanBnXkFtZTcwMzc4NDg2NQ@@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '79%'}, {'Source': 'Metacritic', 'Value': '68/100'}]",68,7.4,"447,664",tt1570728,movie,01 Nov 2011,"$83,300,000",Warner Bros. Pictures,N/A,True
1,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,1964,PG,29 Jan 1964,95 min,Comedy,Stanley Kubrick,"Stanley Kubrick (screenplay), Terry Southern (screenplay), Peter George (screenplay), Peter George (based on the book: ""Red Alert"" by)","Peter Sellers, George C. Scott, Sterling Hayden, Keenan Wynn",An insane general triggers a path to nuclear holocaust that a War Room full of politicians and generals frantically tries to stop.,"English, Russian","UK, USA",Nominated for 4 Oscars. Another 13 wins & 7 nominations.,https://m.media-amazon.com/images/M/MV5BZWI3ZTMxNjctMjdlNS00NmUwLWFiM2YtZDUyY2I3N2MxYTE0XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '8.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '98%'}, {'Source': 'Metacritic', 'Value': '97/100'}]",97,8.4,"427,212",tt0057012,movie,02 Nov 2004,N/A,Sony Pictures,N/A,True
2,"Love, Actually",2003,R,14 Nov 2003,135 min,"Comedy, Drama, Romance",Richard Curtis,Richard Curtis,"Bill Nighy, Gregor Fisher, Rory MacGregor, Colin Firth","Follows the lives of eight very different couples in dealing with their love lives in various loosely interrelated tales all set during a frantic month before Christmas in London, England.","English, Portuguese, French","UK, USA, France",Nominated for 2 Golden Globes. Another 10 wins & 27 nominations.,https://m.media-amazon.com/images/M/MV5BMTY4NjQ5NDc0Nl5BMl5BanBnXkFtZTYwNjk5NDM3._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '64%'}, {'Source': 'Metacritic', 'Value': '55/100'}]",55,7.6,"405,898",tt0314331,movie,27 Apr 2004,"$59,365,105",Universal Pictures,N/A,True
3,Shakespeare in Love,1998,R,08 Jan 1999,123 min,"Comedy, Drama, History, Romance",John Madden,"Marc Norman, Tom Stoppard","Geoffrey Rush, Tom Wilkinson, Steven O'Donnell, Tim McMullan","The world's greatest ever playwright, William Shakespeare, is young, out of ideas and short of cash, but meets his ideal woman and is inspired to write one of his most famous plays.",English,"USA, UK",Won 7 Oscars. Another 57 wins & 87 nominations.,https://m.media-amazon.com/images/M/MV5BM2ZkNjM5MjEtNTBlMC00OTI5LTgyYmEtZDljMzNmNzhiNzY0XkEyXkFqcGdeQXVyNDYyMDk5MTU@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '92%'}, {'Source': 'Metacritic', 'Value': '87/100'}]",87,7.1,"205,961",tt0138097,movie,10 Aug 1999,N/A,Miramax,N/A,True
4,P.S. I Love You,2007,PG-13,21 Dec 2007,126 min,"Comedy, Drama, Romance",Richard LaGravenese,"Richard LaGravenese (screenplay), Steven Rogers (screenplay), Cecelia Ahern (novel)","Hilary Swank, Gerard Butler, Lisa Kudrow, Gina Gershon",A young widow discovers that her late husband has left her 10 messages intended to help ease her pain and start a new life.,English,USA,2 wins & 4 nominations.,https://m.media-amazon.com/images/M/MV5BNTg2MDg4MjI5NV5BMl5BanBnXkFtZTcwMzQ0MDczMw@@._V1_SX300.jpg,"[{'Source': 'Internet Movie Data

## WEB SCRAPING <br>

- For the web scraping portion of the project, you will need to scrape the HTML from your chosen page, parse the HTML to extract the necessary information, and either save the results to a text (txt) file if it is text or into a CSV file if it is tabular data.

In [304]:
url = 'https://www.imdb.com/title/tt0431308/'
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

In [295]:
summary = soup.find_all('div', {'class':'credit_summary_item'})


In [296]:
#director and writer

director = [summary[0].get_text().strip().split('\n')[1]]
director 


['John Madden']

In [297]:
#title from original title OR looking in another class

original_title_box = soup.find_all('div', {'class':'originalTitle'})
title_long = [t.text.split('\xa0')[0] for t in original_title_box]

pattern = re.compile("(.*?)\s*\((.*?)\)")
title = [pattern.match(string).group(1) for string in title_long]
title

#checking f doesn't have original title, use another box
if len(title) == 0:
    title_box = soup.find_all('h1', {'class':''})

    title = [t.text.split('\xa0')[0] for t in title_box]
    title
    


['Shakespeare in Love']

In [298]:
#year

year_box = soup.find_all('h1', {'class':''})
year_long = [y.text.strip().split('\xa0')[1] for y in year_box]

pattern = re.compile(".*?\((.*?)\)")
year = [pattern.match(string).group(1) for string in year_long]
year


['1998']

In [299]:
#rating

rating_box = soup.find_all('div', {'class':'ratingValue'})

rates = [rate.text.strip() for rate in rating_box]

pattern = '\d+\D\d'
rating = [re.findall(pattern,ratio)[0] for ratio in rates]
rating

['7.1']

In [301]:
#all info in one nested list

info = [title, year, rating, director]
info

[['Shakespeare in Love'], ['1998'], ['7.1'], ['John Madden']]

In [302]:
#creating dataframe

df = pd.DataFrame(info)
df

#passing info transposed to have all info about one movie in one line
#will need to add colnames

df_transposed = df.T
df.T

,0
0,Shakespeare in Love
1,1998
2,7.1
3,John Madden


,0,1,2,3
0,Shakespeare in Love,1998,7.1,John Madden


In [45]:
#how to find info tag under tag

#for div in soup.find_all('div', {'class': 'plainlist'}):
 #   for each in div.find_all('a'):  
    #  each.string#get all elements with 'a' tag
     #   href = each.get('href')
      #  print href          #print href
       # print each.string   #print the text in tags
        #print each          #print whole tag

In [10]:
#check if the class is the same for all movies ---> yes

table = soup.find_all('table', {'class':'infobox vevent'})
table

[<table class="infobox vevent" style="width:22em;font-size:90%;"><tbody><tr><th class="summary" colspan="2" style="text-align:center;font-size:125%;font-weight:bold;font-size:110%;font-style:italic;">Crazy, Stupid, Love.</th></tr><tr><td colspan="2" style="text-align:center"><a class="image" href="/wiki/File:CrazyStupidLovePoster.jpg"><img alt="CrazyStupidLovePoster.jpg" class="thumbborder" data-file-height="326" data-file-width="220" decoding="async" height="326" src="//upload.wikimedia.org/wikipedia/en/7/78/CrazyStupidLovePoster.jpg" width="220"/></a><div style="font-size:95%;padding:0.35em 0.35em 0.25em;line-height:1.25em;">Theatrical release poster</div></td></tr><tr><th scope="row" style="white-space:nowrap;padding-right:0.65em;">Directed by</th><td><div class="plainlist"><ul><li><a href="/wiki/Glenn_Ficarra" title="Glenn Ficarra">Glenn Ficarra</a></li><li><a href="/wiki/John_Requa" title="John Requa">John Requa</a></li></ul></div></td></tr><tr><th scope="row" style="white-space:n

In [52]:
#getting colnames, just need to delete the first element

for head in table:
    headers_box = head.find_all('th')
    colnames = [head.text for head in headers_box]
colnames

['Crazy, Stupid, Love.',
 'Directed by',
 'Produced by',
 'Written by',
 'Starring',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office']

In [11]:
#getting rows

for row in table:
    rows = row.find_all('tr')
rows

[<tr><th class="summary" colspan="2" style="text-align:center;font-size:125%;font-weight:bold;font-size:110%;font-style:italic;">Crazy, Stupid, Love.</th></tr>,
 <tr><td colspan="2" style="text-align:center"><a class="image" href="/wiki/File:CrazyStupidLovePoster.jpg"><img alt="CrazyStupidLovePoster.jpg" class="thumbborder" data-file-height="326" data-file-width="220" decoding="async" height="326" src="//upload.wikimedia.org/wikipedia/en/7/78/CrazyStupidLovePoster.jpg" width="220"/></a><div style="font-size:95%;padding:0.35em 0.35em 0.25em;line-height:1.25em;">Theatrical release poster</div></td></tr>,
 <tr><th scope="row" style="white-space:nowrap;padding-right:0.65em;">Directed by</th><td><div class="plainlist"><ul><li><a href="/wiki/Glenn_Ficarra" title="Glenn Ficarra">Glenn Ficarra</a></li><li><a href="/wiki/John_Requa" title="John Requa">John Requa</a></li></ul></div></td></tr>,
 <tr><th scope="row" style="white-space:nowrap;padding-right:0.65em;">Produced by</th><td><div class="p

In [24]:
rows = [row.text.split('\n') for row in rows]
rows

[['Crazy, Stupid, Love.'],
 ['Theatrical release poster'],
 ['Directed byGlenn FicarraJohn Requa'],
 ['Produced bySteve Carell[1]Denise Di Novi'],
 ['Written byDan Fogelman'],
 ['StarringSteve CarellRyan GoslingJulianne MooreEmma StoneMarisa TomeiKevin Bacon'],
 ['Music byChristophe BeckNick Urata'],
 ['CinematographyAndrew Dunn'],
 ['Edited byLee Haxall'],
 ['Productioncompany Carousel Productions[1]Di Novi Pictures '],
 ['Distributed byWarner Bros. Pictures'],
 ['Release date',
  'July\xa019,\xa02011\xa0(2011-07-19) (New York City)',
  'July\xa029,\xa02011\xa0(2011-07-29) (United States)',
  ''],
 ['Running time118 minutes'],
 ['CountryUnited States'],
 ['LanguageEnglish'],
 ['Budget$50 million[2]'],
 ['Box office$145 million[2]']]

In [ ]:
# have two lists

In [53]:
#colnames, delete first item
colnames.pop(0)

'Crazy, Stupid, Love.'

In [55]:
#rows, delete first element
rows.pop(0)

['Crazy, Stupid, Love.']

In [57]:
#rows, delete second element
rows.pop(0)

['Theatrical release poster']

In [54]:
colnames

['Directed by',
 'Produced by',
 'Written by',
 'Starring',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office']

In [58]:
rows

[['Directed byGlenn FicarraJohn Requa'],
 ['Produced bySteve Carell[1]Denise Di Novi'],
 ['Written byDan Fogelman'],
 ['StarringSteve CarellRyan GoslingJulianne MooreEmma StoneMarisa TomeiKevin Bacon'],
 ['Music byChristophe BeckNick Urata'],
 ['CinematographyAndrew Dunn'],
 ['Edited byLee Haxall'],
 ['Productioncompany Carousel Productions[1]Di Novi Pictures '],
 ['Distributed byWarner Bros. Pictures'],
 ['Release date',
  'July\xa019,\xa02011\xa0(2011-07-19) (New York City)',
  'July\xa029,\xa02011\xa0(2011-07-29) (United States)',
  ''],
 ['Running time118 minutes'],
 ['CountryUnited States'],
 ['LanguageEnglish'],
 ['Budget$50 million[2]'],
 ['Box office$145 million[2]']]

In [67]:
for lst, head in zip(rows, colnames):
    for string in lst:
        string.split(head)

['', 'Glenn FicarraJohn Requa']

['', 'Steve Carell[1]Denise Di Novi']

['', 'Dan Fogelman']

['', 'Steve CarellRyan GoslingJulianne MooreEmma StoneMarisa TomeiKevin Bacon']

['', 'Christophe BeckNick Urata']

...

['', '118 minutes']

['', 'United States']

['', 'English']

['', '$50 million[2]']

['', '$145 million[2]']

In [66]:
#unify country, see that have USA in API and united states in wiki.., depending if will need this column..

#colnames = ['Title','Poster','Director', 'Producer', 'Writer', 'Actors', 'Music', 'Cinematography', 'Edited by', 
#            'Prodcution Company', 'Distribution Company', 'Released', 'Runtime', 'Country', 'Language', 'Budget', 'BoxOffice']

#df = pd.DataFrame(rows[1:], columns = colnames)


df = pd.DataFrame(rows[:])
df.index += 1
df

,0,1,2
1,"Crazy, Stupid, Love.",None,None
2,Theatrical release poster,None,None
3,Directed byGlenn FicarraJohn Requa,None,None
4,Produced bySteve Carell[1]Denise Di Novi,None,None
5,Written byDan Fogelman,None,None
6,StarringSteve CarellRyan GoslingJulianne MooreEmma StoneMarisa TomeiKevin Bacon,None,None
7,Music byChristophe BeckNick Urata,None,None
8,CinematographyAndrew Dunn,None,None
9,Edited byLee Haxall,None,None
10,Productioncompany Carousel Productions[1]Di Novi Pictures,None,None
